In [10]:
!conda install -c conda-forge geocoder --yes

Solving environment: done

# All requested packages already installed.



In [11]:
!conda install -c conda-forge folium=0.5.0

Solving environment: done

# All requested packages already installed.



In [28]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import geocoder # import geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [29]:
htmlData  = requests.get( "https://www.health.ny.gov/statistics/cancer/registry/appendix/neighborhoods.htm")
soupData = BeautifulSoup(htmlData.text, 'html.parser')
table_rows = soupData.table.find_all('tr')
table_rows.pop(0)
nyData = pd.DataFrame(columns=['PostalCode','Borough','Neighbourhood'])
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    if len(row)==3:
        borough=row[0]
        neighbourhood=row[1]
        postalcode=row[2]
    if len(row)==2:
        neighbourhood=row[0]
        postalcode=row[1]
    listpostalcode = postalcode.split (",")
    for pc in listpostalcode:
        nyData=nyData.append({'PostalCode':pc,'Borough':borough,'Neighbourhood':neighbourhood}, ignore_index=True)

nyData.head()

,PostalCode,Borough,Neighbourhood
0,10453,Bronx,Central Bronx
1,10457,Bronx,Central Bronx
2,10460,Bronx,Central Bronx
3,10458,Bronx,Bronx Park and Fordham
4,10467,Bronx,Bronx Park and Fordham


In [31]:
geolocator = Nominatim(user_agent="geoapiExercises")
long=[]
lat=[]


for nghbr in nyData.Neighbourhood:
    org_nghbr=nghbr
    if nghbr.find(',') is not None:
        if nghbr.find(',')>0:
            nghbr=nghbr[0:nghbr.find(',')]
    location = geolocator.geocode(nghbr)
    if location is not None:
        latitude = location.latitude
        longitude = location.longitude
    else:
        nghbr=org_nghbr[org_nghbr.find(',')+1:len(org_nghbr)]
        location = geolocator.geocode(nghbr)
        if location is not None:
            latitude = location.latitude
            longitude = location.longitude
        else:
            latitude=0
            longitude=0
    long.append(longitude)
    lat.append(latitude)

print('lists generated')

lists generated


In [32]:
nyData['Latitude']=lat
nyData['Longitude']=long
nyData=nyData[nyData.Latitude != 0.000000]
nyData.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,10453,Bronx,Central Bronx,40.846651,-73.878594
1,10457,Bronx,Central Bronx,40.846651,-73.878594
2,10460,Bronx,Central Bronx,40.846651,-73.878594
3,10458,Bronx,Bronx Park and Fordham,40.861475,-73.890544
4,10467,Bronx,Bronx Park and Fordham,40.861475,-73.890544


In [33]:
CLIENT_ID = 'KUWOR5KHKEMA5XQG4P0HHY10OJTFWTH30NUVROXI0BA4DROV' # your Foursquare ID
CLIENT_SECRET = 'WF3DJ2UHZSB3HH41RQ5ECHXOVZTGGWZLR42F0SKNQGLGWYOE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [42]:
def getNearbyVenues(names, latitudes, longitudes, radius=500,LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        print(url)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [35]:
ny_venues = getNearbyVenues(names=nyData['Neighbourhood'],
                                   latitudes=nyData['Latitude'],
                                   longitudes=nyData['Longitude']
                                  )

ny_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Central Bronx,40.846651,-73.878594,JungleWorld,40.845227,-73.877181,Zoo
1,Central Bronx,40.846651,-73.878594,African Lions,40.847058,-73.878024,Zoo Exhibit
2,Central Bronx,40.846651,-73.878594,Congo Gorilla Forest,40.847774,-73.881604,Zoo
3,Central Bronx,40.846651,-73.878594,Grizzly Corner,40.849023,-73.877739,Zoo Exhibit
4,Central Bronx,40.846651,-73.878594,Giraffe House,40.847875,-73.880127,Zoo Exhibit


In [36]:
ny_onehot=pd.get_dummies(ny_venues[['Venue Category']], prefix="", prefix_sep="")
ny_onehot['Neighbourhood'] = ny_venues['Neighbourhood'] 
fixed_columns = [ny_onehot.columns[-1]] + list(ny_onehot.columns[:-1])
ny_onehot = ny_onehot[fixed_columns]
ny_onehot.head()

,Neighbourhood,Accessories Store,African Restaurant,American Restaurant,Amphitheater,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,...,Warehouse Store,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Central Bronx,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,Central Bronx,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,Central Bronx,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,Central Bronx,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,Central Bronx,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [37]:
ny_grouped = ny_onehot.groupby('Neighbourhood').mean().reset_index()
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = ny_grouped['Neighbourhood']

for ind in np.arange(ny_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ny_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Borough Park,Bank,Pizza Place,Restaurant,Fast Food Restaurant,Café,Hotel,Music Store,Grocery Store,Bistro,Coffee Shop
1,Bronx Park and Fordham,Shoe Store,Pizza Place,Clothing Store,Spanish Restaurant,Coffee Shop,Sandwich Place,Fast Food Restaurant,Pharmacy,Discount Store,Donut Shop
2,Bushwick and Williamsburg,Cocktail Bar,Italian Restaurant,Hotel,Clothing Store,Bowling Alley,Gym / Fitness Center,Bar,Hotel Bar,Flea Market,Waterfront
3,Canarsie and Flatlands,Bus Stop,Liquor Store,Lake,Park,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant
4,Central Bronx,Zoo Exhibit,Zoo,Park,Theme Park Ride / Attraction,Scenic Lookout,Pizza Place,Fast Food Restaurant,Souvenir Shop,Fried Chicken Joint,Garden


In [39]:
htmlData  = requests.get( "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soupData = BeautifulSoup(htmlData.text, 'html.parser')
tableData = pd.DataFrame(columns=['PostalCode','Borough','Neighbourhood'])
table_rows = soupData.table.find_all('tr')
table_rows.pop(0)

for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    tableData=tableData.append({'PostalCode':row[0],'Borough':row[1],'Neighbourhood':row[2].replace('\n','')}, ignore_index=True)

tableData= tableData[tableData.Borough != 'Not assigned']
tableData=tableData.groupby(['PostalCode', 'Borough']).apply(lambda group: ','.join(group['Neighbourhood']))
tableData=tableData.reset_index()
tableData.columns=['PostalCode','Borough','Neighbourhood']
tableData.loc[tableData.Neighbourhood == 'Not assigned','Neighbourhood']= tableData.loc[tableData.Neighbourhood == 'Not assigned','Borough']
tableData.head()


,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [40]:
# The code was removed by Watson Studio for sharing.

In [ ]:
CLIENT_ID = 'GRP5FNZ1YYA4SMGO3RKXYWB30UMWWDZFZZQDWCLBWRZKF1PJ' # your Foursquare ID
CLIENT_SECRET = '1LM3MNB2JUDPKGFN53JRSN0TWPQ2THRXY12N5O2UWK4VSCJD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [44]:


toronto_venues = getNearbyVenues(names=torData['Neighbourhood'],
                                   latitudes=torData['Latitude'],
                                   longitudes=torData['Longitude']
                                  )
toronto_venues.head()

https://api.foursquare.com/v2/venues/explore?&client_id=GRP5FNZ1YYA4SMGO3RKXYWB30UMWWDZFZZQDWCLBWRZKF1PJ&client_secret=1LM3MNB2JUDPKGFN53JRSN0TWPQ2THRXY12N5O2UWK4VSCJD&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100
https://api.foursquare.com/v2/venues/explore?&client_id=GRP5FNZ1YYA4SMGO3RKXYWB30UMWWDZFZZQDWCLBWRZKF1PJ&client_secret=1LM3MNB2JUDPKGFN53JRSN0TWPQ2THRXY12N5O2UWK4VSCJD&v=20180605&ll=43.7845351,-79.16049709999999&radius=500&limit=100
https://api.foursquare.com/v2/venues/explore?&client_id=GRP5FNZ1YYA4SMGO3RKXYWB30UMWWDZFZZQDWCLBWRZKF1PJ&client_secret=1LM3MNB2JUDPKGFN53JRSN0TWPQ2THRXY12N5O2UWK4VSCJD&v=20180605&ll=43.7635726,-79.1887115&radius=500&limit=100
https://api.foursquare.com/v2/venues/explore?&client_id=GRP5FNZ1YYA4SMGO3RKXYWB30UMWWDZFZZQDWCLBWRZKF1PJ&client_secret=1LM3MNB2JUDPKGFN53JRSN0TWPQ2THRXY12N5O2UWK4VSCJD&v=20180605&ll=43.7709921,-79.21691740000001&radius=500&limit=100
https://api.foursquare.com/v2/venues/explore?&client_id=GRP5FNZ1YYA

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,RIGHT WAY TO GOLF,43.785177,-79.161108,Golf Course
2,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place


In [45]:
toronto_onehot=pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted_tor = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted_tor['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted_tor.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted_tor.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Steakhouse,Café,Thai Restaurant,Restaurant,Cosmetics Shop,Bar,Bakery,Asian Restaurant,Breakfast Spot
1,Agincourt,Latin American Restaurant,Breakfast Spot,Skating Rink,Lounge,Diner,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Playground,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Beer Store,Fried Chicken Joint,Fast Food Restaurant,Pharmacy,Pizza Place,Coffee Shop,Sandwich Place,Doner Restaurant,Diner
4,"Alderwood,Long Branch",Pizza Place,Coffee Shop,Pool,Gym,Skating Rink,Pharmacy,Pub,Sandwich Place,Dessert Shop,Dim Sum Restaurant


In [46]:
ny_grouped['City']='New York'
toronto_grouped['City']='Toronto'
grouped_ny_tor= pd.concat([ny_grouped, toronto_grouped], sort=False)
grouped_ny_tor.head()

,Neighbourhood,Accessories Store,African Restaurant,American Restaurant,Amphitheater,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,...,Skating Rink,Smoke Shop,Soccer Field,Stadium,Strip Club,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Video Store
0,Borough Park,0.000000,0.0,0.041667,0.0,0.000000,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Bronx Park and Fordham,0.015625,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Bushwick and Williamsburg,0.000000,0.0,0.010638,0.0,0.010638,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Canarsie and Flatlands,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Central Bronx,0.000000,0.0,0.040000,0.0,0.000000,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

merged_grouped_clustering = grouped_ny_tor.drop('Neighbourhood', 1)
merged_grouped_clustering = merged_grouped_clustering.drop('City', 1)
merged_grouped_clustering=merged_grouped_clustering.fillna(0)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(merged_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 4, 4, 4, 4, 4, 1, 4, 4], dtype=int32)

In [48]:
kmeans.labels_

array([1, 1, 4, 4, 4, 4, 4, 1, 4, 4, 4, 1, 4, 0, 4, 1, 1, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 1, 1, 1, 4, 1, 4, 3, 1, 1, 1, 4, 1,
       1, 4, 1, 1, 4, 4, 4, 1, 3, 1, 4, 1, 1, 4, 1, 4, 4, 4, 1, 1, 4, 1,
       0, 1, 4, 4, 4, 4, 1, 4, 4, 1, 3, 2, 1, 1, 1, 4, 4, 4, 1, 1, 1, 4,
       4, 0, 1, 2, 4, 4, 4, 4, 1, 1, 4, 1, 1, 4, 4, 1, 4, 1, 1, 3, 1, 3,
       4, 0, 1, 1, 4, 4, 1, 1, 1, 1, 4, 4, 1, 4, 3, 4, 1, 1, 1, 1, 1, 4,
       4, 3], dtype=int32)

In [49]:
neighbourhoods_venues_sorted_tor['City']='Toronto'
neighbourhoods_venues_sorted['City']='New York'
neighbourhoods_venues_merged= pd.concat([neighbourhoods_venues_sorted,neighbourhoods_venues_sorted_tor ], sort=False)

# add clustering labels
neighbourhoods_venues_merged.insert(0, 'Cluster Labels', kmeans.labels_)
neighbourhoods_venues_merged
#toronto_merged = torontoData

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
#toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

#toronto_merged.head() # check the last columns!

,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,City
0,1,Borough Park,Bank,Pizza Place,Restaurant,Fast Food Restaurant,Café,Hotel,Music Store,Grocery Store,Bistro,Coffee Shop,New York
1,1,Bronx Park and Fordham,Shoe Store,Pizza Place,Clothing Store,Spanish Restaurant,Coffee Shop,Sandwich Place,Fast Food Restaurant,Pharmacy,Discount Store,Donut Shop,New York
2,4,Bushwick and Williamsburg,Cocktail Bar,Italian Restaurant,Hotel,Clothing Store,Bowling Alley,Gym / Fitness Center,Bar,Hotel Bar,Flea Market,Waterfront,New York
3,4,Canarsie and Flatlands,Bus Stop,Liquor Store,Lake,Park,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,New York
4,4,Central Bronx,Zoo Exhibit,Zoo,Park,Theme Park Ride / Attraction,Scenic Lookout,Pizza Place,Fast Food Restaurant,Souvenir Shop,Fried Chicken Joint,Garden,New York
5,4,Central Brooklyn,Caribbean Restaurant,Juice Bar,Bakery,Furniture / Home Store,Playground,Pharmacy,Gym / Fitness Center,Mobile Phone Shop,Bank,Pizza Place,New York
6,4,Central Harlem,Clothing Store,Cosmetics Shop,Mobile Phone Shop,African Restaurant,Pizza Place,Burger Joint,Theater,Southern / Soul Food Restaurant,Boutique,Deli / Bodega,New York
7,1,Central Queens,Noodle House,Food Court,Chinese Restaurant,Coffee Shop,Shopping Mall,Scandinavian Restaurant,Café,Supermarket,Motorcycle Shop,Spa,New York
8,4,East Harlem,Bakery,Mexican Restaurant,Pizza Place,Latin American Restaurant,Taco Place,Deli / Bodega,Shipping Store,Beer Bar,Gym,Farmers Market,New York
9,4,Flatbush,Caribbean Restaurant,Pizza Place,Deli / Bodega,Fried Chicken Joint,Mobile Phone Shop,Pharmacy,Fast Food Restaurant,Donut Shop,Latin American Restaurant,Southern / Soul Food Restaurant,New York


In [22]:
#nyData[nyData.Latitude != 0.000000]
neighbourhoods_venues_merged[neighbourhoods_venues_merged['Cluster Labels']==0]

,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,City
13,0,Inwood and Washington Heights,Construction & Landscaping,Gas Station,Fast Food Restaurant,Thrift / Vintage Store,Home Service,Zoo Exhibit,Fish Market,Flea Market,Flower Shop,Food Court,New York
30,0,"Del Ray,Keelesdale,Mount Dennis,Silverthorn",Fast Food Restaurant,Restaurant,Sandwich Place,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Toronto
53,0,Hillcrest Village,Pool,Fast Food Restaurant,Golf Course,Dog Run,Mediterranean Restaurant,Drugstore,Diner,Discount Store,Doner Restaurant,Donut Shop,Toronto
75,0,"Rouge,Malvern",Fast Food Restaurant,College Stadium,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Toronto


In [23]:
neighbourhoods_venues_merged[neighbourhoods_venues_merged['Cluster Labels']==1]

,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,City
0,1,Borough Park,Bank,Pizza Place,Restaurant,Fast Food Restaurant,Café,Hotel,Music Store,Grocery Store,Bistro,Coffee Shop,New York
1,1,Bronx Park and Fordham,Shoe Store,Pizza Place,Clothing Store,Spanish Restaurant,Coffee Shop,Sandwich Place,Fast Food Restaurant,Pharmacy,Discount Store,Donut Shop,New York
7,1,Central Queens,Noodle House,Food Court,Chinese Restaurant,Coffee Shop,Shopping Mall,Scandinavian Restaurant,Café,Supermarket,Motorcycle Shop,Spa,New York
11,1,Greenwich Village and Soho,Italian Restaurant,Café,Sushi Restaurant,Cosmetics Shop,American Restaurant,French Restaurant,Cocktail Bar,Ice Cream Shop,Dessert Shop,Salon / Barbershop,New York
15,1,Lower East Side,Mexican Restaurant,Coffee Shop,Cocktail Bar,Bar,Shoe Store,Café,Sandwich Place,Art Gallery,Ice Cream Shop,Chinese Restaurant,New York
16,1,Lower Manhattan,Clothing Store,Pizza Place,Pharmacy,Coffee Shop,Fast Food Restaurant,Supermarket,Gym / Fitness Center,Sandwich Place,Shopping Mall,Sporting Goods Shop,New York
32,1,Upper East Side,Italian Restaurant,Boutique,Coffee Shop,Art Gallery,Art Museum,Cosmetics Shop,French Restaurant,Outdoor Sculpture,Clothing Store,Hotel,New York
33,1,Upper West Side,Italian Restaurant,Wine Bar,Bar,Coffee Shop,Café,American Restaurant,Mediterranean Restaurant,Indian Restaurant,Bakery,Gift Shop,New York
34,1,West Central Queens,Noodle House,Food Court,Chinese Restaurant,Coffee Shop,Shopping Mall,Scandinavian Restaurant,Café,Supermarket,Motorcycle Shop,Spa,New York
0,1,"Adelaide,King,Richmond",Coffee Shop,Steakhouse,Café,Thai Restaurant,Restaurant,Cosmetics Shop,Bar,Bakery,Asian Restaurant,Breakfast Spot,Toronto


In [24]:
neighbourhoods_venues_merged[neighbourhoods_venues_merged['Cluster Labels']==2]

,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,City
41,2,"Emery,Humberlea",Construction & Landscaping,Baseball Field,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Toronto
55,2,"Humber Bay,King's Mill Park,Kingsway Park Sout...",Baseball Field,Yoga Studio,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Toronto


In [25]:
neighbourhoods_venues_merged[neighbourhoods_venues_merged['Cluster Labels']==3]

,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,City
27,3,Southeast Queens,Park,Performing Arts Venue,Zoo Exhibit,Financial or Legal Service,Gas Station,Garden,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,New York
2,3,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Playground,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Toronto
16,3,Caledonia-Fairbanks,Park,Market,Fast Food Restaurant,Women's Store,Comfort Food Restaurant,Comic Shop,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Toronto
40,3,East Toronto,Park,Convenience Store,Dessert Shop,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Toronto
71,3,Parkwoods,BBQ Joint,Food & Drink Shop,Park,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio,Toronto
73,3,Rosedale,Park,Trail,Playground,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Toronto
88,3,"The Kingsway,Montgomery Road,Old Mill North",Park,River,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,Department Store,Toronto
97,3,York Mills West,Park,Bank,Convenience Store,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Toronto


In [26]:
neighbourhoods_venues_merged[neighbourhoods_venues_merged['Cluster Labels']==4]

,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,City
2,4,Bushwick and Williamsburg,Cocktail Bar,Italian Restaurant,Hotel,Clothing Store,Bowling Alley,Gym / Fitness Center,Bar,Hotel Bar,Flea Market,Waterfront,New York
3,4,Canarsie and Flatlands,Bus Stop,Liquor Store,Lake,Park,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,New York
4,4,Central Bronx,Zoo Exhibit,Zoo,Park,Theme Park Ride / Attraction,Scenic Lookout,Pizza Place,Fast Food Restaurant,Souvenir Shop,Fried Chicken Joint,Garden,New York
5,4,Central Brooklyn,Caribbean Restaurant,Juice Bar,Bakery,Furniture / Home Store,Playground,Pharmacy,Gym / Fitness Center,Mobile Phone Shop,Bank,Pizza Place,New York
6,4,Central Harlem,Clothing Store,Cosmetics Shop,Mobile Phone Shop,African Restaurant,Pizza Place,Burger Joint,Theater,Southern / Soul Food Restaurant,Boutique,Deli / Bodega,New York
8,4,East Harlem,Bakery,Mexican Restaurant,Pizza Place,Latin American Restaurant,Taco Place,Deli / Bodega,Shipping Store,Beer Bar,Gym,Farmers Market,New York
9,4,Flatbush,Caribbean Restaurant,Pizza Place,Deli / Bodega,Fried Chicken Joint,Mobile Phone Shop,Pharmacy,Fast Food Restaurant,Donut Shop,Latin American Restaurant,Southern / Soul Food Restaurant,New York
10,4,Greenpoint,Thrift / Vintage Store,Pizza Place,Italian Restaurant,Bakery,Sushi Restaurant,Coffee Shop,Yoga Studio,Donut Shop,French Restaurant,Chinese Restaurant,New York
12,4,High Bridge and Morrisania,Pizza Place,Fried Chicken Joint,Chinese Restaurant,Metro Station,Sandwich Place,Donut Shop,Bakery,Grocery Store,Gym,Mobile Phone Shop,New York
14,4,Kingsbridge and Riverdale,Pizza Place,Sandwich Place,Spanish Restaurant,Donut Shop,Café,Rental Car Location,Breakfast Spot,Supermarket,Discount Store,Candy Store,New York


In [50]:
nyData['Postal Code']=nyData['PostalCode']
neighbourMerged=pd.concat([nyData,torData],sort=False)
neighbourMerged.drop("PostalCode", axis=1,inplace=True)
neighbourhood_venues_merged_longlat=neighbourhoods_venues_merged.join(neighbourMerged.set_index('Neighbourhood'), on='Neighbourhood')
neighbourhood_venues_merged_longlat.head()

,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,City,Borough,Latitude,Longitude,Postal Code
0,1,Borough Park,Bank,Pizza Place,Restaurant,Fast Food Restaurant,Café,Hotel,Music Store,Grocery Store,Bistro,Coffee Shop,New York,Brooklyn,40.633993,-73.996806,11204
0,1,Borough Park,Bank,Pizza Place,Restaurant,Fast Food Restaurant,Café,Hotel,Music Store,Grocery Store,Bistro,Coffee Shop,New York,Brooklyn,40.633993,-73.996806,11218
0,1,Borough Park,Bank,Pizza Place,Restaurant,Fast Food Restaurant,Café,Hotel,Music Store,Grocery Store,Bistro,Coffee Shop,New York,Brooklyn,40.633993,-73.996806,11219
0,1,Borough Park,Bank,Pizza Place,Restaurant,Fast Food Restaurant,Café,Hotel,Music Store,Grocery Store,Bistro,Coffee Shop,New York,Brooklyn,40.633993,-73.996806,11230
1,1,Bronx Park and Fordham,Shoe Store,Pizza Place,Clothing Store,Spanish Restaurant,Coffee Shop,Sandwich Place,Fast Food Restaurant,Pharmacy,Discount Store,Donut Shop,New York,Bronx,40.861475,-73.890544,10458


In [51]:
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[43.653963, -79.387207], zoom_start=1)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(neighbourhood_venues_merged_longlat['Latitude'], neighbourhood_venues_merged_longlat['Longitude'], neighbourhood_venues_merged_longlat['Neighbourhood'], neighbourhood_venues_merged_longlat['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [8]:
htmlDataChicago  = requests.get( "https://www.seechicagorealestate.com/chicago-zip-codes-by-neighborhood.php")
soupDataChicago = BeautifulSoup(htmlDataChicago.text, 'html.parser')
table_rows_chicago = soupDataChicago.table.find_all('tr')
#table_rows_chicago.pop(0)
chicagoData = pd.DataFrame(columns=['PostalCode','Borough','Neighbourhood'])
for tr in table_rows_chicago:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    if row[1]=='Zip Code':
        borough=row[0]
    else:
        neighbourhood=row[0]
        postalcode=row[1]
        listpostalcode = postalcode.split (",")
        for pc in listpostalcode:
            chicagoData=chicagoData.append({'PostalCode':pc,'Borough':borough,'Neighbourhood':neighbourhood}, ignore_index=True)

chicagoData.head()

,PostalCode,Borough,Neighbourhood
0,60611,Downtown,Cathedral District
1,60605,Downtown,Central Station
2,60605,Downtown,Dearborn Park
3,60610,Downtown,Gold Coast
4,60611,Downtown,Gold Coast


In [12]:
geolocator = Nominatim(user_agent="geoapiExercises")
longChicago=[]
latChicago=[]


for nghbr in chicagoData.Neighbourhood:
    org_nghbr=nghbr
    if nghbr.find(',') is not None:
        if nghbr.find(',')>0:
            nghbr=nghbr[0:nghbr.find(',')]
    location = geolocator.geocode(nghbr)
    if location is not None:
        latitude = location.latitude
        longitude = location.longitude
    else:
        nghbr=org_nghbr[org_nghbr.find(',')+1:len(org_nghbr)]
        location = geolocator.geocode(nghbr)
        if location is not None:
            latitude = location.latitude
            longitude = location.longitude
        else:
            latitude=0
            longitude=0
    longChicago.append(longitude)
    latChicago.append(latitude)

print('lists generated')

lists generated


In [20]:
chicagoData['Latitude']=latChicago
chicagoData['Longitude']=longChicago
chicagoData=chicagoData[chicagoData.Latitude != 0.000000]
chicagoData.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,60611,Downtown,Cathedral District,54.950444,-7.740180
1,60605,Downtown,Central Station,50.846548,4.358640
2,60605,Downtown,Dearborn Park,37.258833,-122.318026
3,60610,Downtown,Gold Coast,-28.002373,153.414599
4,60611,Downtown,Gold Coast,-28.002373,153.414599


In [21]:
chicago_venues = getNearbyVenues(names=chicagoData['Neighbourhood'],
                                   latitudes=chicagoData['Latitude'],
                                   longitudes=chicagoData['Longitude']
                                  )

chicago_venues.head()

https://api.foursquare.com/v2/venues/explore?&client_id=KUWOR5KHKEMA5XQG4P0HHY10OJTFWTH30NUVROXI0BA4DROV&client_secret=WF3DJ2UHZSB3HH41RQ5ECHXOVZTGGWZLR42F0SKNQGLGWYOE&v=20180605&ll=54.950443899999996,-7.740180173442905&radius=500&limit=100
https://api.foursquare.com/v2/venues/explore?&client_id=KUWOR5KHKEMA5XQG4P0HHY10OJTFWTH30NUVROXI0BA4DROV&client_secret=WF3DJ2UHZSB3HH41RQ5ECHXOVZTGGWZLR42F0SKNQGLGWYOE&v=20180605&ll=50.8465485,4.3586397&radius=500&limit=100
https://api.foursquare.com/v2/venues/explore?&client_id=KUWOR5KHKEMA5XQG4P0HHY10OJTFWTH30NUVROXI0BA4DROV&client_secret=WF3DJ2UHZSB3HH41RQ5ECHXOVZTGGWZLR42F0SKNQGLGWYOE&v=20180605&ll=37.2588326,-122.3180264&radius=500&limit=100
https://api.foursquare.com/v2/venues/explore?&client_id=KUWOR5KHKEMA5XQG4P0HHY10OJTFWTH30NUVROXI0BA4DROV&client_secret=WF3DJ2UHZSB3HH41RQ5ECHXOVZTGGWZLR42F0SKNQGLGWYOE&v=20180605&ll=-28.0023731,153.4145987&radius=500&limit=100
https://api.foursquare.com/v2/venues/explore?&client_id=KUWOR5KHKEMA5XQG4P0HHY10O

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Cathedral District,54.950444,-7.74018,The Yellow Pepper,54.948210,-7.738285,Restaurant
1,Cathedral District,54.950444,-7.74018,Brewery,54.949648,-7.738022,Pub
2,Cathedral District,54.950444,-7.74018,McGinleys,54.948733,-7.738091,Pub
3,Cathedral District,54.950444,-7.74018,The Cottage Bar,54.951046,-7.736342,Pub
4,Cathedral District,54.950444,-7.74018,Voodoo,54.948877,-7.738126,Nightclub


In [22]:
chicago_onehot=pd.get_dummies(chicago_venues[['Venue Category']], prefix="", prefix_sep="")
chicago_onehot['Neighbourhood'] = chicago_venues['Neighbourhood'] 
fixed_columns = [chicago_onehot.columns[-1]] + list(chicago_onehot.columns[:-1])
chicago_onehot = chicago_onehot[fixed_columns]
chicago_onehot.head()

,Neighbourhood,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Service,American Restaurant,Amphitheater,Animal Shelter,...,Waterfall,Waterfront,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
0,Cathedral District,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Cathedral District,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Cathedral District,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Cathedral District,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Cathedral District,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
chicago_grouped = chicago_onehot.groupby('Neighbourhood').mean().reset_index()
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted_chicago = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted_chicago['Neighbourhood'] = chicago_grouped['Neighbourhood']

for ind in np.arange(chicago_grouped.shape[0]):
    neighbourhoods_venues_sorted_chicago.iloc[ind, 1:] = return_most_common_venues(chicago_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted_chicago.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Addison,Pizza Place,Thai Restaurant,Ice Cream Shop,Mexican Restaurant,Italian Restaurant,Vape Store,Spa,Supermarket,Donut Shop,Lawyer
1,Albany Park,Chinese Restaurant,Grocery Store,Diner,Donut Shop,Fried Chicken Joint,Korean Restaurant,Latin American Restaurant,Cocktail Bar,Hookah Bar,Bakery
2,Algonquin,Intersection,Park,Yoga Studio,Pool Hall,Bakery,Breakfast Spot,Shopping Mall,Seafood Restaurant,River,Pub
3,Alta Vista Terrace,Park,Yoga Studio,Ethiopian Restaurant,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,English Restaurant
4,Andersonville,History Museum,Travel & Transport,American Restaurant,Yoga Studio,Ethiopian Restaurant,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store


In [53]:
chicago_grouped['City']='Chicago'
grouped_chi_tor= pd.concat([chicago_grouped, toronto_grouped], sort=False)
grouped_chi_tor.head()

,Neighbourhood,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Service,American Restaurant,Amphitheater,Animal Shelter,...,Molecular Gastronomy Restaurant,Motel,Poutine Place,Sculpture Garden,Skate Park,Stadium,Strip Club,Swim School,Tanning Salon,Tennis Court
0,Addison,0.0,0.000000,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Albany Park,0.0,0.076923,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Algonquin,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Alta Vista Terrace,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Andersonville,0.0,0.000000,0.0,0.0,0.0,0.0,0.250000,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

merged_grouped_clustering_chi_tor = grouped_chi_tor.drop('Neighbourhood', 1)
merged_grouped_clustering_chi_tor = merged_grouped_clustering_chi_tor.drop('City', 1)
merged_grouped_clustering_chi_tor=merged_grouped_clustering_chi_tor.fillna(0)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(merged_grouped_clustering_chi_tor)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 2, 1, 1, 1, 1, 1, 1], dtype=int32)

In [55]:
neighbourhoods_venues_sorted_chicago['City']='Chicago'
neighbourhoods_venues_merged_chi_tor= pd.concat([neighbourhoods_venues_sorted_chicago,neighbourhoods_venues_sorted_tor ], sort=False)

# add clustering labels
neighbourhoods_venues_merged_chi_tor.insert(0, 'Cluster Labels', kmeans.labels_)
neighbourhoods_venues_merged_chi_tor
#toronto_merged = torontoData

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
#toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

#toronto_merged.head() # check the last columns!

,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,City
0,1,Borough Park,Bank,Pizza Place,Restaurant,Fast Food Restaurant,Café,Hotel,Music Store,Grocery Store,Bistro,Coffee Shop,New York
1,1,Bronx Park and Fordham,Shoe Store,Pizza Place,Clothing Store,Spanish Restaurant,Coffee Shop,Sandwich Place,Fast Food Restaurant,Pharmacy,Discount Store,Donut Shop,New York
2,4,Bushwick and Williamsburg,Cocktail Bar,Italian Restaurant,Hotel,Clothing Store,Bowling Alley,Gym / Fitness Center,Bar,Hotel Bar,Flea Market,Waterfront,New York
3,4,Canarsie and Flatlands,Bus Stop,Liquor Store,Lake,Park,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,New York
4,4,Central Bronx,Zoo Exhibit,Zoo,Park,Theme Park Ride / Attraction,Scenic Lookout,Pizza Place,Fast Food Restaurant,Souvenir Shop,Fried Chicken Joint,Garden,New York
5,4,Central Brooklyn,Caribbean Restaurant,Juice Bar,Bakery,Furniture / Home Store,Playground,Pharmacy,Gym / Fitness Center,Mobile Phone Shop,Bank,Pizza Place,New York
6,4,Central Harlem,Clothing Store,Cosmetics Shop,Mobile Phone Shop,African Restaurant,Pizza Place,Burger Joint,Theater,Southern / Soul Food Restaurant,Boutique,Deli / Bodega,New York
7,1,Central Queens,Noodle House,Food Court,Chinese Restaurant,Coffee Shop,Shopping Mall,Scandinavian Restaurant,Café,Supermarket,Motorcycle Shop,Spa,New York
8,4,East Harlem,Bakery,Mexican Restaurant,Pizza Place,Latin American Restaurant,Taco Place,Deli / Bodega,Shipping Store,Beer Bar,Gym,Farmers Market,New York
9,4,Flatbush,Caribbean Restaurant,Pizza Place,Deli / Bodega,Fried Chicken Joint,Mobile Phone Shop,Pharmacy,Fast Food Restaurant,Donut Shop,Latin American Restaurant,Southern / Soul Food Restaurant,New York


In [56]:
chicagoData['Postal Code']=chicagoData['PostalCode']
neighbourMergedChiTor=pd.concat([chicagoData,torData],sort=False)
neighbourMergedChiTor.drop("PostalCode", axis=1,inplace=True)
neighbourhood_venues_merged_longlat_chitor=neighbourhoods_venues_merged_chi_tor.join(neighbourMergedChiTor.set_index('Neighbourhood'), on='Neighbourhood')
neighbourhood_venues_merged_longlat_chitor.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,City,Borough,Latitude,Longitude,Postal Code
0,1,Addison,Pizza Place,Thai Restaurant,Ice Cream Shop,Mexican Restaurant,Italian Restaurant,Vape Store,Spa,Supermarket,Donut Shop,Lawyer,Chicago,North Side,41.931696,-87.988956,60126
1,1,Albany Park,Chinese Restaurant,Grocery Store,Diner,Donut Shop,Fried Chicken Joint,Korean Restaurant,Latin American Restaurant,Cocktail Bar,Hookah Bar,Bakery,Chicago,North Side,41.971937,-87.716174,60625
2,1,Algonquin,Intersection,Park,Yoga Studio,Pool Hall,Bakery,Breakfast Spot,Shopping Mall,Seafood Restaurant,River,Pub,Chicago,North Side,42.165580,-88.294249,60102
3,2,Alta Vista Terrace,Park,Yoga Studio,Ethiopian Restaurant,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,English Restaurant,Chicago,North Side,39.005388,-77.120534,60613
4,1,Andersonville,History Museum,Travel & Transport,American Restaurant,Yoga Studio,Ethiopian Restaurant,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Chicago,North Side,32.195995,-84.139909,60640


In [57]:
# create map
map_clusters = folium.Map(location=[43.653963, -79.387207], zoom_start=1)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(neighbourhood_venues_merged_longlat_chitor['Latitude'], 
                                  neighbourhood_venues_merged_longlat_chitor['Longitude'], 
                                  neighbourhood_venues_merged_longlat_chitor['Neighbourhood'], 
                                  neighbourhood_venues_merged_longlat_chitor['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters